# Drei gekoppelte Federn

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Ein System von drei gekoppelten Federn mit Dämpfung kann je nach Wahl der Parameter auf ein nicht steifes oder ein steifes Differentialgleichungssystem führen. Damit lässt sich untersuchen, wie unterschiedliche Lösungsverfahren für unterschiedliche Problemtypen geeignet sind.

Dieses Jupyter-Notebook ist in drei Abschnitte aufgeteilt:
1. Zunächst wird die Lösung mit der Defaulteinstellung von `integrate.solve_ivp` aus dem SciPy-Paket berechnet.
2. Anschließend wird die Möglichkeit gegeben, aus den möglichen Lösungsverfahren auszuwählen. Wo möglich, wird auch die Jacobi-Matrix verwendet.
3. Abschließend wird eine Lösung mit Hilfe von Matrizen durchgeführt.

Die zu lösenden dimensionslosen Bewegungsgleichungen lauten

\begin{align}
\frac{\text{d}^2x}{\text{d}t^2} &= -d (x_1-l_1) + (x_2-x_1-l_2) - \kappa_1 v_1 - \kappa_2 (v_1-v_2)\\
\frac{\text{d}^2z}{\text{d}t^2} &= d (1-x_2-l_1) - (x_2-x_1-l_2) - \kappa_1 v_2 + \kappa_2 (v_1-v_2)\,.
\end{align}

## Lösung mit integrate.solve_ivp

### Importanweisungen

Im dritten Teil des Jupyter-Notebooks benötigen wir Funktionen aus dem `linalg`-Modul von NumPy, das wir unter der Abkürzung `LA` importieren.

Aus dem `ipywidgets`-Modul importieren wir zudem die Funktion `interact_manual`, die bei längeren Rechnungen sinnvoll ist. Eine Änderung der Parameter über die graphische Benutzerschnittstelle wird erst wirksam, wenn ein entsprechender Startknopf angeklickt wird.

In [ ]:
import numpy as np
import numpy.linalg as LA
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementierung der Differentialgleichung

Die nachfolgende Funktion ergibt sich aus einer Umschreibung der eingangs angegebenen Differentialgleichungen in vier Differentialgleichungen 1. Ordnung. Sie wird auch im zweiten Abschnitt dieses Jupyter-Notebooks verwendet.

In [ ]:
def dx_dt(t, x, d, kappa_1, kappa_2, l1, l2):
    x1, x2, v1, v2 = x
    dx1_dt = v1
    dx2_dt = v2
    dv1_dt = (-(x1-l1)+d*(x2-x1-l2)
              - kappa_1*v1-kappa_2*(v1-v2))
    dv2_dt = (-d*(x2-x1-l2)+(1-x2-l1)
              - kappa_2*(v2-v1)-kappa_1*v2)
    return dx1_dt, dx2_dt, dv1_dt, dv2_dt

### Lösung des Differentialgleichungssystems

Die numerische Lösung des Differentialgleichungssystems erfolgt hier mit `integrate.solve_ivp` aus dem SciPy-Paket unter Verwendung der Standardeinstellung für den Löser. Es wird also ein Runge-Kutta-Verfahren der Ordnung 5(4) verwendet.

Die Differentialgleichungen enthalten insgesamt fünf Parameter, die mit Hilfe des Arguments `args` übergeben werden und beim Aufruf der Funktion `dx_dt` Verwendung finden.

Zum späteren Vergleich wird neben der Lösung für die beiden schwingenden Punktmassen auch die Anzahl der Aufrufe von `dx_dt` zurückgegeben.

In [ ]:
def trajectory(t_end, n_out, d, kappa_1, kappa_2,
               l1, l2, x0):
    t_values = np.linspace(0, t_end, n_out)
    solution = integrate.solve_ivp(
        dx_dt, (0, t_end), x0, t_eval=t_values,
        args=(d, kappa_1, kappa_2, l1, l2))
    x1, x2 = solution.y[:2, :]
    return solution.t, x1, x2, solution.nfev

### Definition der Anfangsbedingungen und der Systemparameter

Um konsistente Anfangsbedingungen und Systemparameter zu gewährleisten, können diese über die beiden Funktionen `get_initial_conditions` bzw. `get_parameters` erhalten werden. Durch Setzen des Wahrheitswerts `is_stiff` im Argument von `get_parameters` kann zwischen Parametern für steife oder nicht steife Differentialgleichungen gewählt werden.

Falls andere Anfangsbedingungen oder Systemparameter gewünscht werden, können sie hier zentral angepasst werden.

In [ ]:
def get_initial_conditions():
    return [1/3, 2/3, 0.5, 0]

def get_parameters(is_stiff):
    l1, l2 = 0.25, 0.25
    if is_stiff:
        d, kappa_1, kappa_2 = 1000, 0.05, 10
    else:
        d, kappa_1, kappa_2 = 1, 0.05, 0.05
    return d, kappa_1, kappa_2, l1, l2

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des Zeitintervalls
- `n_out`: Anzahl der Zeitpunkte
- `d`: dimensionslose Federkonstante
- `kappa_1`: dimensionslose Dämpfungskonstante für äußere Federn
- `kappa_2`: dimensionslose Dämpfungskonstaten für mittlere Feder

Da die Berechnung etwas länger dauert, soll diese erst erfolgen, wenn die linke Maustaste nach dem Bewegen eines Schiebereglers wieder losgelassen wird. Dies erreichen wir durch die Option `continuous_update=FALSE` im `interact`-Dekorator.

Die noch fehlenden Werte von $l_1$ und $l_2$ erhalten wir durch den Aufruf der Funktion `get_parameters`, wobei die Werte für `d`, `kappa_1`und `kappa_2` ignoriert werden, da diese über die Schieberegler eingestellt werden.

In der graphischen Darstellung wird neben den Schwingungsbewegungen $x_1(t)$ und $x_2(t)$ der beiden Punktmassen auch die zeitliche Veränderung der Länge $x_2(t)-x_1(t)$ der mittleren Feder gezeigt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=100, min=1, max=100, step=0.1,
                   description=r"$t_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=10000, min=100, max=10000,
                   step=100, description=r"$n_\text{out}$"),
               "d":
               widgets.FloatLogSlider(
                   value=1000, base=1000, min=-1, max=3,
                   step=0.1, description="$d$"),
               "kappa_1":
               widgets.FloatLogSlider(
                   value=0.05, base=10, min=-3, max=1,
                   step=0.1, description=r"$\kappa_1$"),
               "kappa_2":
               widgets.FloatLogSlider(
                   value=10, base=10, min=-3, max=1,
                   step=0.1, description=r"$\kappa_2$")
               }

@interact(**widget_dict, continuous_update=False)
def plot_result_1(t_end, n_out, d, kappa_1, kappa_2):
    x0 = get_initial_conditions()
    _, _, _, l1, l2 = get_parameters(False)
    t_values, x1_values, x2_values, count1 = trajectory(
        t_end, n_out, d, kappa_1, kappa_2, l1, l2, x0)

    print(f"{count1} Aufrufe von dx_dt")
    print()

    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(t_values, x1_values, label="$x_1(t)$")
    ax1.plot(t_values, x2_values, label="$x_2(t)$")
    ax1.set_xlabel("$t$")
    ax1.set_ylabel("$x$")
    ax1.legend(loc="lower right")

    ax2.plot(t_values, x2_values-x1_values)
    ax2.set_xlabel("$t$")
    ax2.set_ylabel("$x_2-x_1$")
    ax2.set_xlim((0, 1))

## Lösung mit integrate.solve_ivp unter Verwendung verschiedener Löser

In diesem Abschnitt soll untersucht werden, wie die Zahl der erforderlichen Stützstellen von der Wahl des von `integrate.solve_ivp` verwendeten Lösers abhängt.

### Implementierung der Jacobi-Matrix

Da manche Löser von `integrate.solve_ivp` die Jacobi-Matrix verwenden, stellen wir zu ihrer Berechnung eine Funktion zur Verfügung. Für ein Differentialgleichungssystem der Form

$$\dot x_i = f_i(x_1,\dots,x_n,t)\qquad i=1,\dots, n$$

sind die Matrixelemente der Jacobi-Matrix durch

$$J_{ij} = \frac{\partial f_i}{\partial x_j}$$

gegeben.

In [ ]:
def jacobian(t, x, d, kappa_1, kappa_2, l1, l2):
    x1, x2, v1, v2 = x
    return [[0, 0, 1, 0],
            [0, 0, 0, 1],
            [-1-d, d, -kappa_1-kappa_2, kappa_2],
            [d, -d-1, kappa_2, -kappa_1-kappa_2]]

### Lösung des Differentialgleichungssystems

In der folgenden Funktion werden die explizit über ihren Namen zu übergebenden Argumente in einem Dictionary `kwargs` gesammelt. Dies ermöglicht es uns, bei den entsprechenden Algorithmen die Funktion für die Berechnung der Jacobi-Matrix zu setzen. Im Funktionsaufruf von `solve_ivp` müssen dem Dictionarynamen zwei Sternchen vorangestellt werden.

Neben der Anzahl der Aufrufe von `dx_dt`, die in `solution.nfev` zur Verfügung steht, übergeben wir auch die Anzahl der Aufrufe von `jacobi`, die in `solution.njev` enthalten ist.

In [ ]:
def trajectory_with_method(t_end, n_out, atol, rtol,
                           algorithm, x0, parameters):
    t_values = np.linspace(0, t_end, n_out)
    kwargs = {"t_eval": t_values, "args": parameters,
              "atol": atol, "rtol": rtol,
              "method": algorithm}
    if algorithm in ("BDF", "Radau", "LSODA"):
        kwargs["jac"] = jacobian
    solution = integrate.solve_ivp(dx_dt, (0, t_end),
                                   x0, **kwargs)
    x1, x2 = solution.y[:2, :]
    return solution.t, x1, x2, solution.nfev, solution.njev

### Implementierung der Bedienelemente und graphische Darstellung

Mit Hilfe der graphischen Benutzerschnittstelle lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des Zeitintervalls
- `n_out`: Anzahl der Zeitpunkte
- `atol`: Schranke für absoluten Fehler
- `rtol`: Schranke für relativen Fehler
- `algorithm`: zu verwendender Löser
- `is_stiff`: Auswahl eines Parametersatzes für steifes oder nicht steifes Differentialgleichungssystem

Neben der graphischen Darstellung der Bewegung der Punktmassen und der Zeitabhängigkeit der Länge der mittleren Feder wird auch die Anzahl der Aufrufe von `dx_dt` und `jacobian` ausgegeben. Damit lässt sich der Rechenaufwand für die verschiedenen Löser vergleichen.

In [ ]:
new_widgets = {"atol":
               widgets.FloatLogSlider(
                   value=1e-5, base=10, min=-10, max=-3,
                   step=0.1, description="atol"),
               "rtol":
               widgets.FloatLogSlider(
                   value=1e-5, base=10, min=-10, max=-3,
                   step=0.1, description="rtol"),
               "algorithm":
               widgets.RadioButtons(
                   options=["RK45", "RK23", "DOP853",
                            "Radau", "BDF", "LSODA"],
                   value="RK45",
                   description="Lösungsalgorithmus"),
               "is_stiff":
               widgets.RadioButtons(
                   options=[("nicht steif", False),
                            ("steif", True)],
                   description="Steifheit")
               }
widget_dict.update(new_widgets)

@interact(**widget_dict)
def plot_result_2(t_end, n_out, atol, rtol,
                  algorithm, is_stiff):
    x0 = get_initial_conditions()
    parameters = get_parameters(is_stiff)

    t_values, x1_values, x2_values, count1, count2 = (
        trajectory_with_method(t_end, n_out, atol, rtol,
                               algorithm, x0, parameters))

    print(f"{count1} Aufrufe von dx_dt")
    print(f"{count2} Aufrufe von jac")
    print("")

    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(t_values, x1_values, label="$x_1(t)$")
    ax1.plot(t_values, x2_values, label="$x_2(t)$")
    ax1.set_xlabel("$t$")
    ax1.set_ylabel("$x$")
    ax1.legend(loc="lower right")

    ax2.plot(t_values, x2_values-x1_values)
    ax2.set_xlabel("$t$")
    ax2.set_ylabel("$x_2-x_1$")
    ax2.set_xlim((0, 1))

## Lösung mit Hilfe von Matrizen

Wenn man die Auslenkung

$$\Delta \boldsymbol{x} = \boldsymbol{x} - \boldsymbol{x}^{(0)}$$

aus der Gleichgewichtslage $\boldsymbol{x}^{(0)}$ betrachtet, lassen sich die oben angegebenen Bewegungsgleichungen in der Form

$$\frac{\text{d}}{\text{d} t} \Delta \boldsymbol{x} = \hat M  \Delta \boldsymbol{x}$$

schreiben, wobei $\hat M$ eine konstante Matrix ist. Die Lösung dieser Differentialgleichung ist durch

$$\Delta \boldsymbol{x}(t) = \exp(\hat{M} t) \Delta\boldsymbol{x}(0)$$

gegeben.

### Berechnung der Zeitentwicklungsmatrix

Die nachfolgende Funktion berechnet

$$\exp\left( \hat{M} t\right) = \hat{T} \exp\left(\hat{D}t\right) \hat{T}^{-1}\,,$$

wobei $\hat{T}$ die Matrix mit den Eigenvektoren und $\hat{D}$ die Diagonalmatrix mit den Eigenwerten von $\hat{M}$ ist.

Der Umweg über die Eigendarstellung ist erforderlich, da die Exponentialfunktion aus dem NumPy-Paket die Exponentialfunktion der einzelnen Matrixelemente und nicht der Matrix als solcher berechnet.

In [ ]:
def propagator(t, m):
    evals, evecs = LA.eig(m)
    diag = np.diag(np.exp(evals*t))
    exp_m = evecs @ diag @ LA.inv(evecs)
    return exp_m.real

### Lösung der Differentialgleichung

Zur Vorbereitung der Lösung mit Hilfe von Matrizen werden zunächst die Gleichgewichtslagen $x_1^{(0)}$ und $x_2^{(0)}$ sowie der anfängliche Auslenkungsvektor $\Delta\boldsymbol{x}^{(0)}$ berechnet und den Variablen `x1_eq`, `x2_eq` bzw. `delta_x_0` zugewiesen. Zudem wird die konstante Matrix $\hat M$ erstellt.

Der zentrale Rechenschritt erfolgt zu Beginn der `for`-Schleife, wo der Zeitentwickungsoperator für einen gegebenen Zeitpunkt berechnet und mit den anfänglichen Auslenkungen multipliziert wird. Anschließend werden die absoluten Auslenkungen berechnet und in `x1_values` und `x2_values` abgespeichert. Zudem werden die Eigenwerte von $\hat M$ zurückgegeben.


In [ ]:
def trajectory_matrices(t_end, n_out, d,
                        kappa_1, kappa_2, l1, l2, x0):
    x1_eq = (l1+d*(1-l2)) / (2*d+1)
    x2_eq = 1 - x1_eq
    delta_x_0 = (np.array(x0)
                 - np.array((x1_eq, x2_eq, 0, 0)))

    m = np.array([[0, 0, 1, 0],
                  [0, 0, 0, 1],
                  [-1-d, d, -kappa_1-kappa_2, kappa_2],
                  [d, -1-d, kappa_2, -kappa_1-kappa_2]]
                 )

    t_values = np.linspace(0, t_end, n_out)
    x1_values = np.empty_like(t_values)
    x2_values = np.empty_like(t_values)
    for idx, t in enumerate(t_values):
        delta_x = propagator(t, m) @ delta_x_0
        x1_values[idx] = delta_x[0]+x1_eq
        x2_values[idx] = delta_x[1]+x2_eq
    return t_values, x1_values, x2_values, LA.eigvals(m)

### Implementierung der Bedienelemente und graphische Darstellung

Mit Hilfe der graphischen Benutzerschnittstelle lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des Zeitintervalls
- `n_out`: Anzahl der Zeitpunkte
- `is_stiff`: Auswahl eines Parametersatzes für steifes oder nicht steifes Differentialgleichungssystem

Neben der graphischen Darstellung der Zeitabhängigkeit der Auslenkungen der beiden Punktmassen sowie der Länge der mittleren Feder werden auch die Eigenwerte von $\hat M$ und deren Beträge ausgegeben, um einen Vergleich von steifen und nicht steifen Differentialgleichungssystemen zu ermöglichen.

In [ ]:
interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_result_3(t_end, n_out, is_stiff):
    x0 = get_initial_conditions()
    d, kappa_1, kappa_2, l1, l2 = get_parameters(is_stiff)

    t_values, x1_values, x2_values, ew = (
        trajectory_matrices(t_end, n_out, d, kappa_1,
                            kappa_2, l1, l2, x0))

    print("Eigenwerte von M:")
    for value in ew:
        print(value)
    print()
    print("und deren Beträge:")
    for value in ew:
        print(abs(value))
    print()

    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(t_values, x1_values, label="$x_1(t)$")
    ax1.plot(t_values, x2_values, label="$x_2(t)$")
    ax1.set_xlabel("$t$")
    ax1.set_ylabel("$x$")
    ax1.legend(loc="lower right")

    ax2.plot(t_values, x2_values-x1_values)
    ax2.set_xlabel("$t$")
    ax2.set_ylabel("$x_2-x_1$")
    ax2.set_xlim((0, 1))